# Image Pixelization with Differential Privacy

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import random
import numpy as np
import sys
from PIL import Image

In [3]:
data_path = "/content/drive/MyDrive/dp-pix/data/samples" #사용자의 데이터가 저장된 경로
save_path = "/content/drive/MyDrive/dp-pix/results_image/" #결과이미지가 저장될 경로

In [4]:
sys.path.insert(0, 'drive/MyDrive/dp-pix')

In [5]:
from src import util as iu
from src import dp_pixel as dp
from src import dataset as db
from src.resize import Resize
from src.pixelate import Pixelate

In [6]:
resize_f = Resize.pad_image
pixelate_f = Pixelate.pytorch

In [7]:
def load(dataset):
    images = dataset.load_idx_images()
    iu.display_image_grid(images)
    return images

In [8]:
def load_all(dataset):
    images = dataset.load_all_images()
    iu.display_image_grid(images)
    return images

In [9]:
def dp_pixelate_images(images, target_h, target_w, m, eps, noise_func):
    noisy_images = [dp.dp_pixelate(I,target_h, target_w, m, eps, noise_func, resize_f=resize_f,
                                   pixelate_f=pixelate_f) for I in images]
    iu.display_image_grid(noisy_images)
    return noisy_images

In [10]:
def vary_eps(images, choice_idx, target_h, target_w, m, noise_func, start, end, step):
    Select_img = images[choice_idx-1]
    eps_vals = np.arange(start,end+step,step)
    noisy_images = [dp.dp_pixelate(Select_img, target_h, target_w, m, eps, noise_func, resize_f=resize_f,
                                  pixelate_f=pixelate_f) for eps in eps_vals]
    titles=["{:.1f}".format(eps) for eps in eps_vals]
    iu.display_image_grid(noisy_images, titles=titles)
    return noisy_images

In [11]:
# save dp-pix img
def save_img(image_list, save_path):
    for i in range(len(image_list)):
      uint8_arr=(image_list[i]).astype(np.uint8)
      arr2img=Image.fromarray(uint8_arr)
      arr2img.save(save_path + str(i+1)+'th_sample.png',"PNG")
      print("image save success!")

# Sample Dataset

In [12]:
file_idx=1 # 몇번째 파일을 불러올지

## Set the parameters

In [19]:
target_h = 50 # 사용자 지정
target_w = 50
m = 8
eps = 10
noise_func="laplace"

In [ ]:
samples = load(db.Dataset(data_path, file_idx))

In [ ]:
all_samples = load_all(db.Dataset(data_path, file_idx))

## DP Pixelated Images

In [ ]:
# one img
result_imgs=dp_pixelate_images(samples, target_h, target_w, m, eps, noise_func)

In [ ]:
save_img(result_imgs, save_path)  #사용자가 save_path 수정 가능

In [ ]:
# all img
result_all_imgs=dp_pixelate_images(all_samples, target_h, target_w, m, eps, noise_func)

In [ ]:
save_img(result_all_imgs, save_path)  #사용자가 save_path 수정 가능

## Decreasing $eps$ 10 $\rightarrow$ 1 by 1

In [ ]:
decrese_eps=vary_eps(samples, 1, target_h, target_w, m, noise_func, 10, 1, -1)

In [ ]:
save_img(decrese_eps, save_path) #사용자가 save_path 수정 가능

In [ ]:
#!pip install lpips

## LPIPS Score

In [ ]:
import lpips
from PIL import Image
import torch
from torchvision import transforms

In [ ]:
def calculate_lpips_similarity(image1, image2):
    # 이미지 불러오기
    #image1 = Image.open(image_path1).convert("RGB")
    #image2 = Image.open(image_path2).convert("RGB")

    # 이미지를 텐서로 변환
    transform = transforms.ToTensor()
    image1_tensor = transform(image1)
    image2_tensor = transform(image2)

    # LPIPS 모델 불러오기
    loss_fn = lpips.LPIPS(net="alex")

    # 이미지 유사도 계산
    similarity = 1.0 - loss_fn(image1_tensor, image2_tensor).item()

    return similarity

In [ ]:
samples_uint8_arr=(samples[0]).astype(np.uint8)
samples_arr2img=Image.fromarray(samples_uint8_arr)

In [ ]:
results_uint8_arr=(result_imgs[0]).astype(np.uint8)
results_arr2img=Image.fromarray(results_uint8_arr)

In [ ]:
# 이미지 파일 경로 설정
# image_path1 = "path/to/your/image1.jpg"
# image_path2 = "path/to/your/image2.jpg"

# 유사도 계산
similarity_score = calculate_lpips_similarity(samples_arr2img, results_arr2img)

print(f"LPIPS Similarity: {similarity_score}")